# Pre-joining Features to Main Training Datasets

This notebook aims to pre-join the processed sea surface temparature grid and precipitation grid to the dataset of each vessel. As this will reduce the computation power and time greatly. (deprecated - migrate to join_df.py and feature_engineering.py)

In [1]:
import pandas as pd
import numpy as np

sst_df = pd.read_csv('sst_grid.csv', index_col=0)
precip_df = pd.read_csv('precip_grid.csv', index_col=0)

sst_df["time_bnds"] = pd.to_datetime(sst_df["time_bnds"]).dt.to_period('M')
precip_df["time"] = pd.to_datetime(precip_df["time"]).dt.to_period('M')

C:\Users\USER\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
path = 'drifting_longlines.csv'

fishing_df = pd.read_csv(path)
fishing_df = fishing_df[fishing_df['is_fishing'] > -0.5]
fishing_df['is_fishing'] = [0 if x < 0.3 else 1 for x in fishing_df['is_fishing']]
fishing_df = fishing_df[['is_fishing', 'lat', 'lon', 'course', 'speed', 'timestamp', 'distance_from_shore', 'distance_from_port', 'mmsi', 'source']]

def custom_season(x): 
    return np.round(int(x)/3)

def custom_round(x): 
    return 0.5 + np.floor(float(x))

fishing_df['adjust_lat'] = fishing_df['lat'].apply(lambda x: custom_round(x))
fishing_df['adjust_lon'] = fishing_df['lon'].apply(lambda x: custom_round(x))
fishing_df["adjust_time_date"] = pd.to_datetime(fishing_df['timestamp'], unit='s')
fishing_df["adjust_time"] = pd.to_datetime(fishing_df["adjust_time_date"]).dt.to_period('M')
fishing_df["doy"] = fishing_df["adjust_time_date"].dt.dayofyear
fishing_df["dow"] = fishing_df["adjust_time_date"].dt.dayofweek
fishing_df["season"] = fishing_df["adjust_time_date"].dt.month.apply(lambda x: custom_season(x))
fishing_df["year"] = fishing_df["adjust_time_date"].dt.year
fishing_df["month"] = fishing_df["adjust_time_date"].dt.month
fishing_df["day"] = fishing_df["adjust_time_date"].dt.day
fishing_df["hour"] = fishing_df["adjust_time_date"].dt.hour
fishing_df = fishing_df.drop(columns=['timestamp', 'adjust_time_date'])

In [3]:
df_all = pd.merge(fishing_df, sst_df,  how='left', \
                  left_on=['adjust_lat','adjust_lon', 'adjust_time'], \
                  right_on = ['lat','lon', 'time_bnds'])
df_all = pd.merge(df_all, precip_df,  how='left', \
                  left_on=['adjust_lat','adjust_lon', 'adjust_time'], \
                  right_on = ['lat','lon', 'time'])
df_all = df_all.drop(columns=['adjust_time', 'adjust_lon', 'adjust_lat', 'time', 'lat', 'lon', 'lat_y','lon_y', 'time_bnds'])
df_all.to_csv('new_' + path)
df_all

,is_fishing,lat_x,lon_x,course,speed,distance_from_shore,distance_from_port,mmsi,source,doy,dow,season,year,month,day,hour,sst,precip
0,1,18.608902,-17.205967,162.800003,8.2,111123.328125,147593.812500,1.263956e+13,dalhousie_longliner,154,5,2.0,2012,6,2,17,28.758749,NaN
1,1,18.839836,-19.453779,333.899994,5.0,329078.531250,362363.843750,1.263956e+13,dalhousie_longliner,171,1,2.0,2012,6,19,8,28.859999,NaN
2,1,19.276720,-17.323196,260.100006,0.7,86831.046875,196301.062500,1.263956e+13,dalhousie_longliner,236,3,3.0,2012,8,23,7,29.183750,NaN
3,1,18.852455,-17.255033,43.400002,4.2,98881.343750,166465.140625,1.263956e+13,dalhousie_longliner,244,4,3.0,2012,8,31,5,29.351250,NaN
4,1,19.101482,-17.147205,333.500000,7.0,74247.757812,177480.859375,1.263956e+13,dalhousie_longliner,249,2,3.0,2012,9,5,6,29.189999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219736,0,15.686500,-26.522484,48.000000,9.3,195926.296875,218814.203125,2.812058e+14,crowd_sourced,360,4,4.0,2014,12,26,18,28.724998,NaN
219737,0,15.695100,-26.513317,45.500000,9.5,194512.578125,217406.718750,2.812058e+14,crowd_sourced,360,4,4.0,2014,12,26,18,28.724998,NaN
219738,0,15.713583,-26.493151,46.099998,8.8,191685.140625,214592.031250,2.812058e+14,crowd_sourced,360,4,4.0,2014,12,26,18,28.724998,NaN
219739,0,15.748650,-26.454216,46.299999,9.1,185343.781250,208189.515625,2.812058e+14,crowd_sourced,360,4,4.0,2014,12,26,19,28.724998,NaN


In [4]:
df_all.dropna()

,is_fishing,lat_x,lon_x,course,speed,distance_from_shore,distance_from_port,mmsi,source,doy,dow,season,year,month,day,hour,sst,precip
348,1,58.017879,-173.384689,57.400002,11.1,372125.28125,3.927562e+05,5.139444e+13,dalhousie_longliner,157,1,2.0,2012,6,5,15,11.508750,2.782652
349,1,58.021347,-173.368378,132.899994,8.2,370352.90625,3.909904e+05,5.139444e+13,dalhousie_longliner,157,1,2.0,2012,6,5,15,11.508750,2.782652
374,1,56.314812,-166.433578,360.000000,8.2,343424.25000,4.317023e+05,5.139444e+13,dalhousie_longliner,193,2,2.0,2012,7,11,1,16.876999,2.708316
375,1,56.337833,-166.453781,181.600006,3.2,340680.62500,4.369618e+05,5.139444e+13,dalhousie_longliner,193,2,2.0,2012,7,11,5,16.876999,2.708316
376,1,56.287521,-166.433411,1.900000,2.6,344260.34375,4.267693e+05,5.139444e+13,dalhousie_longliner,193,2,2.0,2012,7,11,10,16.876999,2.708316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218881,1,58.959785,-17.464716,172.500000,10.6,509811.96875,1.081266e+06,2.812058e+14,crowd_sourced,293,0,3.0,2014,10,20,3,8.570000,2.856602
218882,0,58.896534,-17.446367,173.899994,11.1,500336.59375,1.095248e+06,2.812058e+14,crowd_sourced,293,0,3.0,2014,10,20,4,8.570000,2.856602
218883,0,58.835880,-17.425367,170.600006,10.6,490772.00000,1.108561e+06,2.812058e+14,crowd_sourced,293,0,3.0,2014,10,20,4,8.570000,2.856602
218884,0,58.772415,-17.405300,170.600006,11.1,481727.75000,1.122548e+06,2.812058e+14,crowd_sourced,293,0,3.0,2014,10,20,5,8.570000,2.856602
